In [8]:
# required modules
import pandas as pd
import numpy as np 
import matplotlib
%matplotlib inline
matplotlib.style.use('default')
import requests
import json

In [2]:
# data and google api key setup
google_api_access_key = open('../.env/.google_api_access_key').read()

In [2]:
# let's use our great df, we created in Grid Generator notebook
df_grid_points_final = pd.read_excel('../raw_data/raw_data_geneva_grid_points.xlsx', usecols='B:D') 
df_grid_points_final

,lat_long,lat,long
0,"46.20151452321782,6.111010044545912",46.201515,6.111010
1,"46.20213626535293,6.111010044545912",46.202136,6.111010
2,"46.20337972851531,6.111010044545912",46.203380,6.111010
3,"46.20400144954259,6.111010044545912",46.204001,6.111010
4,"46.20462316353395,6.111010044545912",46.204623,6.111010
...,...,...,...
3240,"46.1990274843178,6.1756887450025175",46.199027,6.175689
3241,"46.19778392265196,6.176587060286637",46.197784,6.176587
3242,"46.19840570700288,6.176587060286637",46.198406,6.176587
3243,"46.1990274843178,6.176587060286637",46.199027,6.176587


In [3]:
# convert grid points to numpy array for an efficient iteration which is far more efficient than iterrows()
grid_points_array = df_grid_points_final[['lat','long']].to_numpy()
print(grid_points_array.shape)
grid_points_array

(3245, 2)


array([[46.20151452,  6.11101004],
       [46.20213627,  6.11101004],
       [46.20337973,  6.11101004],
       ...,
       [46.19840571,  6.17658706],
       [46.19902748,  6.17658706],
       [46.19902748,  6.17748538]])

# Google Places Nearby Search

In [ ]:
# Sample coordinate
lat, lng = grid_points_array[1][0], grid_points_array[1][1]
print(lat, lng)

# Sample response / Google nearby search
base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json?'

params = {
    'keyword':'restaurant',
    'location':f'{lat},{lng}',
    'radius':100,
    'region':'ch',
    'key':google_api_access_key
}

response_sample = requests.get(base_url, params=params).json()
df_sample = pd.json_normalize(response_sample['results'])\
        [['name',
          'place_id',
          'types',
          'price_level',
          'rating',
          'user_ratings_total',
          'vicinity',
          'plus_code.global_code',
          'geometry.location.lat',
          'geometry.location.lng']]
df_sample

In [83]:
%%time
#this is my test to check the git (Tchucky)
base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json?' # base url

df_restaurants = pd.DataFrame()

for row in grid_points_array:
    lat, lng = row[0], row[1]
    radius = 100 #meters

    params = {
        'keyword':'restaurant',
        'location':f'{lat},{lng}',
        'radius':radius,
        'region':'ch',
        'key':google_api_access_key
    }
    
    try:
        response_basic = requests.get(base_url, params=params).json()

        # create a temporarily dataframe to hold response
        df_temp= pd.json_normalize(response_basic['results'])\
            [['name',
              'place_id',
              'types',
              'price_level',
              'rating',
              'user_ratings_total',
              'vicinity',
              'plus_code.global_code',
              'geometry.location.lat',
              'geometry.location.lng']]

        # concat and remove duplicates (beware drop_duplicates do not work due to lists values)
        drop_ref = df_temp.drop(columns=['types']).columns.to_list()
        df_restaurants = pd.concat([df_restaurants, df_temp], axis=0)\
            .drop_duplicates(drop_ref)\
            .reset_index(drop=True)
        
    except:
        continue
print(f'last_row: {row}')

last_row: [46.19902748  6.17748538]
CPU times: user 1min 31s, sys: 5.62 s, total: 1min 37s
Wall time: 44min 17s


In [84]:
df_restaurants

,name,place_id,types,price_level,rating,user_ratings_total,vicinity,plus_code.global_code,geometry.location.lat,geometry.location.lng
0,Café Restaurant Palma,ChIJX68uubZkjEcRKKxpZ7OrgqU,"[restaurant, food, point_of_interest, establis...",NaN,3.7,141.0,"Av. d'Aïre 93bis, Genève",8FR86437+XJ,46.204921,6.114022
1,Restaurant La Tentation,ChIJd6kh67ZkjEcRC1IQourQOyo,"[restaurant, food, point_of_interest, establis...",2.0,4.2,158.0,"Av. d'Aïre 91, Genève",8FR86447+3X,46.205283,6.114920
2,Nénuphar Café | Restaurant,ChIJt8c02LFkjEcRcbEARQLCYQo,"[restaurant, food, point_of_interest, establis...",1.0,4.7,63.0,"Rte de l'Usine-à-Gaz 10, Vernier",8FR86445+C6,46.206069,6.108121
3,Restaurant Les Jardins du Rhône,ChIJV8uySFxljEcRn7XpIO3zZX0,"[restaurant, food, point_of_interest, establis...",NaN,0.0,0.0,"Av. d'Aïre 89, Genève",8FR86448+36,46.205248,6.115503
4,Canteen De La Residence Gardens Du Rhône,ChIJW8zU8rZkjEcR_Ke8t7c7hVc,"[restaurant, food, point_of_interest, establis...",NaN,5.0,1.0,"Av. d'Aïre 89, Genève",8FR86448+54,46.205387,6.115347
...,...,...,...,...,...,...,...,...,...,...
1111,MAMS,ChIJr8wyS6BljEcRc6tPa0MXzt8,"[restaurant, cafe, bakery, food, point_of_inte...",NaN,4.9,13.0,"Rte de Chêne 67, Chêne-Bougeries",8FR855XG+V2,46.199666,6.174996
1112,Allianz Cinema Genève,ChIJFVY2_UNljEcRkTweEUHkbXw,"[movie_theater, bar, restaurant, point_of_inte...",NaN,4.3,221.0,"Quai Gustave-Ador 87, Genève",8FR8656C+F5,46.211129,6.170376
1113,KATANA Sushi,ChIJpYWlTUlljEcR1wQr6MUxT4M,"[restaurant, food, point_of_interest, establis...",NaN,4.6,160.0,"Av. de la Gare des Eaux-Vives 22a, Genève",8FR86529+QF,46.201944,6.168695
1114,Dunia,ChIJS2Mt5E5ljEcRRTLNjqD4jBY,"[furniture_store, restaurant, point_of_interes...",NaN,4.4,74.0,"Chem. Frank-Thomas 12, Genève",8FR8653C+65,46.203026,6.170426


In [89]:
df_restaurants.to_csv('../raw_data/raw_data_google_api.csv', encoding='utf-8-sig')

# Google Places Details Search

In [6]:
# Sample response / Google Places Details search
base_url = 'https://maps.googleapis.com/maps/api/place/details/json?'

place_id = 'ChIJX68uubZkjEcRKKxpZ7OrgqU'

# field dimension to investigate
params = {
    'place_id':place_id,
    'key':google_api_access_key
}

# filter also columns 
response_details_sample = requests.get(base_url, params=params).json()
df_details_sample = pd.json_normalize(response_details_sample)
df_details_sample

,html_attributions,status,result.address_components,result.adr_address,result.business_status,result.formatted_address,result.formatted_phone_number,result.geometry.location.lat,result.geometry.location.lng,result.geometry.viewport.northeast.lat,...,result.plus_code.global_code,result.rating,result.reference,result.reviews,result.types,result.url,result.user_ratings_total,result.utc_offset,result.vicinity,result.website
0,[],OK,"[{'long_name': '93bis', 'short_name': '93bis',...","<span class=""street-address"">Av. d&#39;Aïre 93...",OPERATIONAL,"Av. d'Aïre 93bis, 1203 Genève, Switzerland",022 345 84 48,46.204931,6.114021,46.206337,...,8FR86437+XJ,3.7,ChIJX68uubZkjEcRKKxpZ7OrgqU,"[{'author_name': 'Amin Masroor', 'author_url':...","[restaurant, point_of_interest, food, establis...",https://maps.google.com/?cid=11926283550252969000,141,120,"Avenue d'Aïre 93bis, Genève",http://palmarestaurant.ch/


In [12]:
response_details_sample

{'html_attributions': [],
 'result': {'address_components': [{'long_name': '93bis',
    'short_name': '93bis',
    'types': ['street_number']},
   {'long_name': "Avenue d'Aïre",
    'short_name': "Av. d'Aïre",
    'types': ['route']},
   {'long_name': 'Genève',
    'short_name': 'Genève',
    'types': ['locality', 'political']},
   {'long_name': 'Genève',
    'short_name': 'Genève',
    'types': ['administrative_area_level_2', 'political']},
   {'long_name': 'Genève',
    'short_name': 'GE',
    'types': ['administrative_area_level_1', 'political']},
   {'long_name': 'Switzerland',
    'short_name': 'CH',
    'types': ['country', 'political']},
   {'long_name': '1203', 'short_name': '1203', 'types': ['postal_code']}],
  'adr_address': '<span class="street-address">Av. d&#39;Aïre 93bis</span>, <span class="postal-code">1203</span> <span class="locality">Genève</span>, <span class="country-name">Switzerland</span>',
  'business_status': 'OPERATIONAL',
  'formatted_address': "Av. d'Aïre 9